In [49]:
# https://towardsdatascience.com/combining-numerical-and-text-features-in-deep-neural-networks-e91f0237eea4
"""combine meta data with text in keras"""

'combine meta data with text in keras'

In [ ]:
import fasttext
import fasttext.util
ft = fasttext.load_model(embedding_loc)

In [189]:
import spacy
import gensim
nlp = spacy.load('en')
import pandas as pd
import numpy as np
import re
import demoji
from googletrans import Translator
translator = Translator()
import wordninja

In [190]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/constraint_Hindi_Train - Sheet1.csv"
data = pd.read_csv(file_path)
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [191]:
# making labels
from keras.utils import to_categorical
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [int(x) for x in y]
y = to_categorical(y)
y[:10]

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [192]:
hindi_stop_words = ["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
len(hindi_stop_words)

225

In [193]:
def feature_analysis(tweet):
    # url
    urls = re.findall(r"(?:https?\://)\S+", tweet)
    ## remove url from tweet
    tweet = re.sub(r"(?:https?\://)\S+", "", tweet)
    
    #cleaning 
    pat = re.compile(r"([',./(;_`:|\-)!])")
    tweet = pat.sub("", tweet)
    tweet = tweet.rstrip("\n")
    tweet = re.sub('\n', ' ', tweet)
    tweet = tweet.replace('\u200b', '')
    tweet = tweet.replace('\u200c', '')
    tweet = tweet.replace('\u200d', '')
    
    # hashtag
    hashtag_words = re.findall(r'#([^\s]+)', tweet)
    ## remove hashtag
    tweet = re.sub(r'#([^\s]+)', "", tweet)
    
    #mentions
    mention_words = re.findall(r'@([^\s]+)', tweet)
    ## remove mentions
    tweet = re.sub(r'@([^\s]+)', '', tweet)
    
    #print(translator.translate('Jankari', dest='hi').text)
    # english words
    english_words = [word for word in tweet.split() if re.search(r'[a-zA-Z]', word)]
    ## remove english words
    tweet = " ".join([word for word in tweet.split() if word not in english_words])
    #tweet = " ".join([word if word not in english_words else translator.translate(" ".join(wordninja.split(word)), dest='hi').text for word in tweet.split()])
    #f(x) if condition else g(x) for x in sequence
    emojis = demoji.findall_list(tweet, desc = True)
    tweet = demoji.replace(tweet,"")
    tweet = " ".join([word for word in tweet.split() if word not in hindi_stop_words])
    return tweet, urls, hashtag_words, mention_words, english_words, emojis

In [194]:
from tqdm import tqdm

In [195]:
all_tweets= []
for tweet in tqdm(data['Post']):
    all_tweets.append(feature_analysis(tweet))

100%|██████████| 5728/5728 [00:07<00:00, 782.00it/s]


In [196]:
embedding_loc = "/home/ayush/Documents/personal/research/aaai/hostility_classification/fastText/cc.hi.300.bin"

In [197]:
X = []
for tweet in all_tweets:
    X.append(ft.get_sentence_vector(tweet[0]))

In [198]:
df = pd.DataFrame(data=X)
df.shape

(5728, 300)

In [199]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.01, random_state=42)

In [200]:
# X_train = df[:5500]
# y_train = y[:5500]

# X_test = df[5500:]
# y_test = y[5500:]

In [201]:
y_train.shape

(5670, 2)

In [202]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

In [203]:
# model = Sequential()
# model.add(Embedding(vocab_length, 20, input_length= 300))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))

In [204]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [217]:
import keras
my_callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              verbose=1, mode='auto')]

In [227]:
import tensorflow as tf
opt = tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.1, nesterov=True, name="SGD")

In [239]:
input2 = Input(shape=(300,))
dense_layer_1 = Dense(100, activation='relu')(input2)
dense_layer_2 = Dense(100, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(100, activation='relu')(dense_layer_2)
output = Dense(2, activation='sigmoid')(dense_layer_3)

model = Model(inputs=input2, outputs=output)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc',f1_m])

In [240]:
history = model.fit(X_train, y_train, batch_size=16, epochs=170, verbose=2,validation_split=0.2)

Epoch 1/170
284/284 - 0s - loss: 0.6913 - acc: 0.5353 - f1_m: 0.4826 - val_loss: 0.6920 - val_acc: 0.5088 - val_f1_m: 0.5088
Epoch 2/170
284/284 - 0s - loss: 0.6887 - acc: 0.5386 - f1_m: 0.5392 - val_loss: 0.6906 - val_acc: 0.5088 - val_f1_m: 0.5088
Epoch 3/170
284/284 - 0s - loss: 0.6867 - acc: 0.5386 - f1_m: 0.5383 - val_loss: 0.6892 - val_acc: 0.5088 - val_f1_m: 0.5088
Epoch 4/170
284/284 - 0s - loss: 0.6848 - acc: 0.5386 - f1_m: 0.5385 - val_loss: 0.6878 - val_acc: 0.5088 - val_f1_m: 0.5088
Epoch 5/170
284/284 - 0s - loss: 0.6827 - acc: 0.5386 - f1_m: 0.5385 - val_loss: 0.6856 - val_acc: 0.5088 - val_f1_m: 0.5088
Epoch 6/170
284/284 - 0s - loss: 0.6802 - acc: 0.5386 - f1_m: 0.5390 - val_loss: 0.6828 - val_acc: 0.5088 - val_f1_m: 0.5088
Epoch 7/170
284/284 - 0s - loss: 0.6770 - acc: 0.5386 - f1_m: 0.5391 - val_loss: 0.6783 - val_acc: 0.5088 - val_f1_m: 0.5146
Epoch 8/170
284/284 - 0s - loss: 0.6729 - acc: 0.5401 - f1_m: 0.5428 - val_loss: 0.6731 - val_acc: 0.5414 - val_f1_m: 0.5504


284/284 - 0s - loss: 0.3142 - acc: 0.8618 - f1_m: 0.8626 - val_loss: 0.3369 - val_acc: 0.8651 - val_f1_m: 0.8659
Epoch 67/170
284/284 - 0s - loss: 0.3142 - acc: 0.8653 - f1_m: 0.8651 - val_loss: 0.3349 - val_acc: 0.8660 - val_f1_m: 0.8658
Epoch 68/170
284/284 - 0s - loss: 0.3152 - acc: 0.8635 - f1_m: 0.8643 - val_loss: 0.3300 - val_acc: 0.8686 - val_f1_m: 0.8671
Epoch 69/170
284/284 - 0s - loss: 0.3078 - acc: 0.8679 - f1_m: 0.8676 - val_loss: 0.3641 - val_acc: 0.8545 - val_f1_m: 0.8545
Epoch 70/170
284/284 - 0s - loss: 0.3033 - acc: 0.8717 - f1_m: 0.8717 - val_loss: 0.3431 - val_acc: 0.8633 - val_f1_m: 0.8642
Epoch 71/170
284/284 - 0s - loss: 0.3091 - acc: 0.8627 - f1_m: 0.8631 - val_loss: 0.3290 - val_acc: 0.8713 - val_f1_m: 0.8711
Epoch 72/170
284/284 - 0s - loss: 0.3080 - acc: 0.8664 - f1_m: 0.8664 - val_loss: 0.3586 - val_acc: 0.8536 - val_f1_m: 0.8535
Epoch 73/170
284/284 - 0s - loss: 0.3048 - acc: 0.8710 - f1_m: 0.8709 - val_loss: 0.3592 - val_acc: 0.8527 - val_f1_m: 0.8539
Epoch

Epoch 131/170
284/284 - 0s - loss: 0.2041 - acc: 0.9204 - f1_m: 0.9207 - val_loss: 0.3214 - val_acc: 0.8774 - val_f1_m: 0.8774
Epoch 132/170
284/284 - 0s - loss: 0.2078 - acc: 0.9217 - f1_m: 0.9220 - val_loss: 0.3201 - val_acc: 0.8810 - val_f1_m: 0.8803
Epoch 133/170
284/284 - 0s - loss: 0.1990 - acc: 0.9235 - f1_m: 0.9230 - val_loss: 0.3258 - val_acc: 0.8783 - val_f1_m: 0.8787
Epoch 134/170
284/284 - 0s - loss: 0.1952 - acc: 0.9239 - f1_m: 0.9242 - val_loss: 0.3513 - val_acc: 0.8765 - val_f1_m: 0.8770
Epoch 135/170
284/284 - 0s - loss: 0.2010 - acc: 0.9224 - f1_m: 0.9225 - val_loss: 0.3331 - val_acc: 0.8721 - val_f1_m: 0.8721
Epoch 136/170
284/284 - 0s - loss: 0.1900 - acc: 0.9292 - f1_m: 0.9294 - val_loss: 0.4323 - val_acc: 0.8210 - val_f1_m: 0.8206
Epoch 137/170
284/284 - 0s - loss: 0.2006 - acc: 0.9198 - f1_m: 0.9202 - val_loss: 0.3358 - val_acc: 0.8836 - val_f1_m: 0.8838
Epoch 138/170
284/284 - 0s - loss: 0.1959 - acc: 0.9242 - f1_m: 0.9242 - val_loss: 0.3144 - val_acc: 0.8783 - v

In [242]:
model.evaluate(X_test, y_test, verbose=2)

2/2 - 0s - loss: 0.5185 - acc: 0.8793 - f1_m: 0.8798


[0.5185034275054932, 0.8793103694915771, 0.8798075914382935]

In [208]:
from sklearn.metrics import accuracy_score

In [209]:
accuracy_score(y_test,predictions)

NameError: name 'predictions' is not defined

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, predictions))

In [ ]:
# testing data

In [ ]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/Constraint_Hindi_Valid - Sheet1.csv"
data = pd.read_csv(file_path)

In [ ]:
# making labels
from keras.utils import to_categorical
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [int(x) for x in y]
y = to_categorical(y)
y[:10]

In [ ]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(feature_analysis(tweet))

In [ ]:
X_text = []
for tweet in all_tweets:
    X_text.append(ft.get_sentence_vector(tweet[0]))

In [ ]:
df = pd.DataFrame(data=X_text)
df.shape

In [ ]:
model.evaluate(df, y, verbose=2)

In [ ]:
# from sklearn.decomposition import PCA
# #pca=PCA(n_components=50)

# X=pca.transform(X)

In [ ]:
predictions=clf.predict(X)

In [ ]:
accuracy_score(y,predictions)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y, predictions))